In [ ]:
%cd /app

from cjio import cityjson
import cjio
import json
import os
import pyproj
from pyproj import Transformer
import pandas as pd
from src.parse_city_json import remove_empty_geometries, remove_empty_egid, preprocess_json

source_crs = "EPSG:2056"
target_crs = "EPSG:4326"

transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

In [ ]:
preprocess_json("/app/data/swissBUILDINGS3D_3-0_1047-43.json", "/app/data/swissBUILDINGS3D_3-0_1047-43.json")

In [ ]:
cm = cityjson.load("/app/data/swissBUILDINGS3D_3-0_1047-43.json", transform=True)

In [ ]:
buildings = remove_empty_egid(cm.get_cityobjects(type="building"))
building_parts = remove_empty_egid(cm.get_cityobjects(type=["buildingpart"]))

In [ ]:
type(buildings)

In [ ]:
for b in buildings:
    data = buildings[b]

In [ ]:
vertices = data.get_vertices()

def get_minimum_height(vertex_list):
    min_height = 1e9

    for v in vertex_list:
        if v[2] < min_height:
            min_height = v[2]
    return min_height




In [ ]:
geom = data.geometry[0]

roofs = geom.get_surfaces("RoofSurface")

for r in roofs:
    rf = roofs[r]
    t = geom.get_surface_boundaries(rf)
t

In [ ]:
geom.get_surface_boundaries(roofs[1])

In [ ]:
roofs = data.geometry[0].get_surfaces("RoofSurface")
roof_boundaries = []

for r in roofs.values():
    roof_boundaries.append(data.geometry[0].get_surface_boundaries(r))
roof_boundaries

In [ ]:
for x in roof_boundaries[0]:
    x

In [ ]:
data.attributes.get("")

In [ ]:
def process_boundary_generator(generator: iter, transformer: Transformer):

    transformed_coordinates = []
    for x in generator:
        y = transformer.itransform(x[0])
        transformed_coordinates.append([yy for yy in y])
    
    return transformed_coordinates


def extract_roof_geometries(co: cjio.models.CityObject, transformer: Transformer):

    geometry_data = {}
    
    egid = co.attributes.get("EGID")
    if egid is None:
        raise ValueError("CityObject does not have an 'EGID' attribute")
    
    geometry_data = []

    for i, geom in enumerate(co.geometry):
        geom_type = geom.type
        roofs = geom.get_surfaces("RoofSurface")
        roofs_transformed = []
        for r in roofs:
            if isinstance(r, int):
                rf = roofs[r]
                boundary = geom.get_surface_boundaries(rf)
            else:
                boundary = r
            roofs_transformed.append(process_boundary_generator(boundary, transformer))

        roof_dict = {}
        roof_dict["type"] = geom_type
        roof_dict["coordinates"] = roofs_transformed
        geometry_data.append(roof_dict)

    print(f"{i+1} records processed")

    return {egid: geometry_data}

In [ ]:
building_roof_coordinates = {}
for b in buildings:
    data = buildings[b]
    building_roof_coordinates.update(extract_roof_geometries(data, transformer))

In [ ]:
# save to json
with open("/app/data/building_roof_coordinates.json", 'w') as f:
    json.dump(building_roof_coordinates, f, indent=2)